[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Image Processing with Python

## Image Binarization by Optimization on Graphs

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 03/10/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/ImageProcessingPython/0002SciKitImageBasics.ipynb)

In [1]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import jit, njit

# Image Processing
import skimage as ski

# Machine Learning


# Miscellaneous
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, List, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatRangeSlider, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [2]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [3]:
# Constants



In [4]:
# Course Packages


In [5]:
# Auxiliary Functions

from AuxFun import ImageGradient



## Image Binarization

The concept of Image Binarization is creating a binary image from a given (Usually _grayscale_) image.  
The operation is common in Text Binarization as a pre processing for [Optical Character Recognition](https://en.wikipedia.org/wiki/Optical_character_recognition) (OCR).

This notebook uses a Binary [Markov Random Field](https://en.wikipedia.org/wiki/Markov_random_field) (MRF) to build a graph.  
The optimization problem is solved by a [Graph Cut](https://en.wikipedia.org/wiki/Graph_cuts_in_computer_vision): [_Min Cut_ / _Max Flow_](https://en.wikipedia.org/wiki/Max-flow_min-cut_theorem).

* <font color='brown'>(**#**)</font> The algorithm borrows some ideas from [A Laplacian Energy for Document Binarization](https://ieeexplore.ieee.org/document/6065266).
* <font color='brown'>(**#**)</font> An alternative approach to solve the problem using optimization is given by [Automatic Image Enhancement of Images of Scanned Documents (Auto Whitening)](https://dsp.stackexchange.com/questions/50329).

### Binary MRF and Min Cut

A binary MRF is given by:

$$ \Epsilon \left( B \right) = \sum_{i, j} {L}_{i, j}^{0} + \sum_{i, j} {L}_{i, j}^{1} + \sum_{i, j} \sum_{m, n \in \mathcal{N} \left( i, j \right)} C \left( {B}_{i, j} , {B}_{m, n} \right) $$

Where

 - $B$ - Binary image where ${B}_{i, j} \in \left\{ 0, 1 \right\}$.
 - ${L}_{i, j}^{0}$ - The cost of assigning the value $0$ to the pixel at $i, j$.
 - ${L}_{i, j}^{1}$ - The cost of assigning the value $1$ to the pixel at $i, j$.
 - $C \left( {B}_{i, j} , {B}_{m, n} \right)$ - The cost of pixels having same / different values from their neighborhood.

In [V. Kolmogorov, R. Zabin - What Energy Functions Can Be Minimized via Graph Cuts](https://ieeexplore.ieee.org/document/1262177) defined the conditions for having a closed form solution to the above.  
The condition is called _Sub Modularity_ which requires $C \left( 0, 1 \right) + C \left( 1, 0 \right) \geq C \left( 0, 0 \right) + C \left( 1, 1 \right)$.

The concept os solving such problem is by creating a graph with 2 additional nodes which stands for the label `0` and `1`:

![](https://i.imgur.com/SoxBpBo.png)
<!-- ![](https://i.postimg.cc/26JSTWyZ/image.png) -->

The weights defined by the cost of having labels `0` / `1` and the inter connection between pixels as given by $C \left( \cdot , \cdot \right)$.  
Given the graph, the Min Cut can be calculated and its solution is the minimizer of the MRF objective.


* <font color='brown'>(**#**)</font> An efficient algorithm is given by [Y. Boykov, V. Kolmogorov - An Experimental Comparison of Min Cut / Max Flow Algorithms for Energy Minimization in Vision](https://ieeexplore.ieee.org/document/1316848).
* <font color='brown'>(**#**)</font> Useful books on the subject:
    - [Olivier Lezoray](https://lezoray.users.greyc.fr), Leo Grady - Image Processing and Analysis with Graphs.
    - [Daphne Koller](https://ai.stanford.edu/~koller), Nir Friedman - Probabilistic Graphical Models.

In [6]:
# Parameters

# imgUrl = 'https://i.imgur.com/0OVu3QL.jpeg' #<! Certificate of Arrival for Berta Werner
imgUrl = 'https://i.postimg.cc/2S6SWmCJ/Sample003.jpg' #<! Certificate of Arrival for Berta Werner

## Generate Data


The input image is [Certificate of Arrival for Berta Werner](https://commons.wikimedia.org/wiki/File:Certificate_of_Arrival_for_Berta_Werner._-_NARA_-_282038.jpg).

In [7]:
# Generate / Load the Data

mI = ski.io.imread(imgUrl)


In [ ]:
# Display Data

hF, hA = plt.subplots(figsize = (8, 6))
hA.imshow(mI, cmap = 'gray')
hA.set_title('Certificate of Arrival for Berta Werner');

## Analysis

### Generate the MRF Matrices

The MRF will be given as a function of the [Image Laplacian](https://en.wikipedia.org/wiki/Discrete_Laplace_operator):

 - ${L}_{i, j}^{0} = {\nabla}^{2} {I}_{i, j}$
 - ${L}_{i, j}^{1} = -{\nabla}^{2} {I}_{i, j}$

This matches that valleys will have low cost to have `0` value.

The function $C$ is given by:

$$
C 
$$

In [9]:
# Image Laplacian 

#===========================Fill This===========================#
# 1. Convert the image to `Float64` in [0, 1] range.
# 2. Calculate the Laplace Operator.
# !! You may find `ski.util.img_as_float64()` and `ski.filters.laplace()` useful.

mI = ski.util.img_as_float64(mI) #<! Image in range [0, 1], as Float 64
mL = ski.filters.laplace(mI) #<! Laplacian of the Image

#===============================================================#

In [ ]:
# Display Laplacian 

hF, hA = plt.subplots(figsize = (8, 6))
hA.imshow(mL, cmap = 'gray', vmin = mL.min(), vmax = mL.max())
hA.set_title('The Image Laplacian');


In [11]:
# Image Gradient

#===========================Fill This===========================#
# 1. Calculate the image gradient vector.
# !! You may find `ImageGradient()` useful.

mIx, mIy = ImageGradient(mI) #<! Gradient Vector

#===============================================================#

In [ ]:
# Display Gradient 

hF, vHa = plt.subplots(nrows = 1, ncols = 2, figsize = (8, 6))

hA = vHa.flat[0]
hA.imshow(mIx, cmap = 'gray', vmin = mIx.min(), vmax = mIx.max())
hA.set_title('The `x` Gradient');

hA = vHa.flat[1]
hA.imshow(mIy, cmap = 'gray', vmin = mIx.min(), vmax = mIx.max())
hA.set_title('The `y` Gradient');

In [13]:
# Image Edge Detection 

#===========================Fill This===========================#
# 1. Extract binary images of the edges of the image.
# !! You may find `ski.feature.canny()` useful.

mE = ski.feature.canny(mI) #<! Edge Image

#===============================================================#

In [ ]:
# Display Edge 

hF, hA = plt.subplots(figsize = (8, 6))
hA.imshow(mE, cmap = 'gray', vmin = mL.min(), vmax = mL.max())
hA.set_title('The Edge Detector');

In [ ]:
# 

mCH = ~((mE[:-1, :-1] & (mIy > 0)) | (mE[1:, :-1] & (mIy <= 0)))
mCV = ~((mE[:-1, :-1] & (mIx > 0)) | (mE[:-1, 1:] & (mIx <= 0)))